In [192]:
import os

In [193]:
%pwd

'C:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [194]:
os.chdir("C:/Users/Dheer/OneDrive/Desktop/MLOps/MediWatch_project")

In [195]:
%pwd

'C:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [196]:
# You can find the Password in Dagshub under Settings -> Tokens tab. Use the Default access token -> copy
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/dheeraj24797/MediWatch_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="dheeraj24797"
os.environ["MLFLOW_TRACKING_PASSWORD"]="66e0128e3659638b468a1827a8e4621ed8ed7cd3"

In [197]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [198]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories, save_json

In [199]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_params
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = Path,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/dheeraj24797/MediWatch_project.mlflow",
           
        )

        return model_evaluation_config

In [200]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
# Need to change these.

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred, pred_proba=None):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average="weighted", zero_division=0)
        recall = recall_score(actual, pred, average="weighted", zero_division=0)
        f1 = f1_score(actual, pred, average="weighted", zero_division=0)
        roc_auc = roc_auc_score(actual, pred_proba) if pred_proba is not None else None
        return accuracy, precision, recall, f1, roc_auc

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_labels = model.predict(test_x)
            predicted_proba = model.predict_proba(test_x)[:, 1]

            accuracy, precision, recall, f1, roc_auc = self.eval_metrics(
                test_y, predicted_labels, predicted_proba
            )

            # Save metrics locally
            scores = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
                "roc_auc": roc_auc,
            }
            save_json(self.config.metric_file_name, scores)

            # Log params and metrics to MLflow
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            if roc_auc is not None:
                mlflow.log_metric("roc_auc", roc_auc)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBClassifierModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [202]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-09 13:25:29,047: INFO: common: Raw YAML content from WindowsPath('config/config.yaml'): {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/dheeraj24797/MediWatch_project/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/diabetic_data.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/diabetic_data.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifac

TypeError: expected str, bytes or os.PathLike object, not type